# Raising The Village Program Monitoring Analysis
**Analyst:** Mugume Martin
**Date:** November 14, 2025


This notebook analyzes household-level program data from the Raising The Village initiative.
The objective is to assess household adoption of WASH, Agriculture, and VSLA practices, and to monitor program performance across regions, districts, and clusters.

In [1]:

import os
os.chdir(r'E:\MUGUME\Raising village')

In [2]:
ls

 Volume in drive E has no label.
 Volume Serial Number is 0DD0-07C4

 Directory of E:\MUGUME\Raising village

11/14/2025  09:13 AM    <DIR>          .
11/14/2025  09:07 AM    <DIR>          ..
11/11/2025  01:23 AM            96,888 household_list.xlsx
11/11/2025  01:18 AM           378,489 households_coaching_visits data 2.xlsx
11/11/2025  01:12 AM           900,937 households_coaching_visits.xlsx
11/11/2025  01:19 AM           643,823 households_training_attendance..xlsx
11/14/2025  09:10 AM    <DIR>          scheduledwrittenassessmentdataanalystprogrammonitori
11/14/2025  09:07 AM         1,659,039 scheduledwrittenassessmentdataanalystprogrammonitori.zip
               5 File(s)      3,679,176 bytes
               3 Dir(s)  155,721,109,504 bytes free


In [12]:
import pandas as pd
import glob
from datetime import datetime

household_list = pd.read_excel('household_list.xlsx')


households_coaching_visits2 = pd.read_excel('households_coaching_visits data 2.xlsx')

households_coaching_visits = pd.read_excel('households_coaching_visits.xlsx')

households_training_attendance = pd.read_excel('households_training_attendance..xlsx')




In [13]:
print("household_list:", household_list.columns.tolist())
print("households_coaching_visits data 2:", households_coaching_visits2.columns.tolist())
print("households_coaching_visits:", households_coaching_visits.columns.tolist())
print("households_training_attendance:", households_training_attendance.columns.tolist())


household_list: ['region_name', 'district_name', 'cluster_name', 'village_name', 'household_id', 'deleted_or_deactivated_household']
households_coaching_visits data 2: ['region_name', 'district_name', 'cluster_name', 'village_name', 'user_type', 'household_id', 'hhid', 'latrine_present', 'latrine_cover', 'paper_leaves_present', 'latrine_floored', 'latrine_envir_human_excreta_free', 'latrine_door', 'latrine_walled', 'latrine_roof_leak_proof', 'tippy_present', 'tippy_water_fill', 'soap_ash_present', 'latrine_envir_veg_free', 'cloth_hang_line_present', 'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present', 'bathroom_drainage', 'compound_clean', 'dishrack_present', 'dishrack_double_pres', 'compost_present', 'rec_num_compost', 'standard_size_compost', 'compost_fill_correctly', 'lqd_manure_prod', 'org_presticide_prod', 'compound_gardening', 'biodegrade_waste_mgt', 'mulch_practice', 'vegetables', 'staple_crops', 'post_harvest_storage', 'water_control_practice', 'bbw_man

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from docx import Document
from docx.shared import RGBColor, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
import warnings
warnings.simplefilter("ignore", category=FutureWarning)


BASE_DIR = Path(r"E:\MUGUME\Raising village")  
FILES = {
    "households": BASE_DIR / "household_list.xlsx",
    "coaching1": BASE_DIR / "households_coaching_visits.xlsx",
    "coaching2": BASE_DIR / "households_coaching_visits data 2.xlsx",
    "training": BASE_DIR / "households_training_attendance..xlsx"
}
OUTPUT_DIR = BASE_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

REPORT_PATH = OUTPUT_DIR / "Mugume_program_monitoring_report_complete.docx"
DASH_APP_PATH = OUTPUT_DIR / "app_mugume_dash_complete.py"


CHALLENGES_LOG = []

def log_challenge(challenge, solution):
    """Track challenges and solutions for documentation"""
    CHALLENGES_LOG.append({"challenge": challenge, "solution": solution})
    print(f"[CHALLENGE] {challenge}")
    print(f"[SOLUTION] {solution}\n")


def normalize_cols(df):
    df = df.copy()
    df.columns = (df.columns.astype(str)
                  .str.strip()
                  .str.lower()
                  .str.replace(' ', '_')
                  .str.replace('-', '_'))
    return df

def choose_and_unify_hhid(df, dataset_name=""):
    """
    Deterministic: prefer 'hhid' if present and non-null, else 'household_id'.
    Creates 'hhid_final' column.
    """
    df = df.copy()
    if 'hhid' in df.columns and df['hhid'].notna().sum() > 0:
        df['hhid_final'] = df['hhid']
        chosen = 'hhid'
    elif 'household_id' in df.columns:
        df['hhid_final'] = df['household_id']
        chosen = 'household_id'
    else
        id_cols = [c for c in df.columns if 'id' in c and 'date' not in c]
        if id_cols:
            df['hhid_final'] = df[id_cols[0]]
            chosen = id_cols[0]
            log_challenge(
                f"In {dataset_name}: Both 'hhid' and 'household_id' missing",
                f"Using fallback column '{id_cols[0]}' as household identifier"
            )
        else:
            df['hhid_final'] = np.nan
            chosen = 'none'
            log_challenge(
                f"In {dataset_name}: No ID column found",
                "Created hhid_final with NaN values - requires manual review"
            )
    
    if dataset_name and chosen in ['hhid', 'household_id']:
        null_count = df['hhid_final'].isna().sum()
        if null_count > 0:
            log_challenge(
                f"In {dataset_name}: Found {null_count} null values in {chosen}",
                f"These records will be excluded from household-level analysis"
            )
    
    return df

def excel_serial_to_datetime(val):
    """
    Convert Excel serial to datetime when needed. Handles numeric serials and strings.
    Excel epoch origin 1899-12-30.
    """
    try:
        if pd.isna(val):
            return pd.NaT
        # numeric
        if isinstance(val, (int, float, np.number)) or str(val).replace('.', '', 1).isdigit():
            return pd.to_datetime(float(val), unit='d', origin='1899-12-30', errors='coerce')
        # otherwise try parse
        return pd.to_datetime(val, errors='coerce')
    except Exception:
        return pd.NaT

def safe_mean_row(df, cols):
    if not cols:
        return pd.Series([np.nan]*len(df), index=df.index)
    return df[cols].apply(pd.to_numeric, errors='coerce').mean(axis=1, skipna=True)


print("="*80)
print("PROGRAM MONITORING DATA ANALYSIS - RAISING THE VILLAGE")
print("="*80)
print("\nLoading files from:", BASE_DIR)
for name, path in FILES.items():
    if not path.exists():
        raise FileNotFoundError(f"Expected {name} at {path} but file not found. Update BASE_DIR or filenames.")
    print(f"  ✓ {name}: {path.name}")

hh = normalize_cols(pd.read_excel(FILES['households'], engine='openpyxl'))
c1_raw = pd.read_excel(FILES['coaching1'], engine='openpyxl')
c2_raw = pd.read_excel(FILES['coaching2'], engine='openpyxl')
training = normalize_cols(pd.read_excel(FILES['training'], engine='openpyxl'))
print("\n✓ Files loaded successfully.")


print("\n" + "="*80)
print("IDENTIFYING COACHING COMPONENT COLUMNS (H:AB for WASH, AC:AP for AGRI)")
print("="*80)

# Excel columns H:AB = indices 7:28 (0-indexed), AC:AP = indices 28:42
WASH_START_IDX = 7   # Column H
WASH_END_IDX = 28    # Column AB (exclusive in slice)
AGRI_START_IDX = 28  # Column AC
AGRI_END_IDX = 42    # Column AP (exclusive in slice)

# Use c1_raw to identify columns (before normalization)
print(f"\nChecking column positions in '{FILES['coaching1'].name}':")
print(f"  Column H (index {WASH_START_IDX}): {c1_raw.columns[WASH_START_IDX]}")
print(f"  Column AB (index {WASH_END_IDX-1}): {c1_raw.columns[WASH_END_IDX-1]}")
print(f"  Column AC (index {AGRI_START_IDX}): {c1_raw.columns[AGRI_START_IDX]}")
print(f"  Column AP (index {AGRI_END_IDX-1}): {c1_raw.columns[AGRI_END_IDX-1]}")


c1 = normalize_cols(c1_raw)
c2 = normalize_cols(c2_raw)

wash_cols = c1.columns[WASH_START_IDX:WASH_END_IDX].tolist()
agri_cols = c1.columns[AGRI_START_IDX:AGRI_END_IDX].tolist()


vsla_cols = [c for c in c1.columns if 'vsla' in c.lower() and 'participation' in c.lower()]
if not vsla_cols:
    
    vsla_cols = [c for c in c1.columns if 'vsla' in c.lower()]
    if vsla_cols:
        log_challenge(
            "VSLA Participation column not explicitly named",
            f"Using column(s): {vsla_cols}"
        )
    else:
        log_challenge(
            "No VSLA column found in coaching dataset",
            "VSLA score will be calculated as NaN"
        )

print(f"\n✓ Identified {len(wash_cols)} WASH columns (H:AB)")
print(f"✓ Identified {len(agri_cols)} Agriculture columns (AC:AP)")
print(f"✓ Identified {len(vsla_cols)} VSLA column(s)")


col_mapping = pd.DataFrame({
    'Component': ['WASH']*len(wash_cols) + ['Agriculture']*len(agri_cols) + ['VSLA']*len(vsla_cols),
    'Column_Name': wash_cols + agri_cols + vsla_cols
})
col_mapping.to_csv(OUTPUT_DIR / "Mugume_column_mapping.csv", index=False)
print(f"\n✓ Saved column mapping to Mugume_column_mapping.csv")


print("\n" + "="*80)
print("STANDARDIZING HOUSEHOLD IDENTIFIERS")
print("="*80)

hh = choose_and_unify_hhid(hh, "household_list")
c1 = choose_and_unify_hhid(c1, "coaching_visits_1")
c2 = choose_and_unify_hhid(c2, "coaching_visits_2")
training = choose_and_unify_hhid(training, "training_attendance")

print("\n✓ Created hhid_final in all datasets")
print(f"  Household list: {hh['hhid_final'].nunique()} unique households")
print(f"  Coaching 1: {c1['hhid_final'].nunique()} unique households")
print(f"  Coaching 2: {c2['hhid_final'].nunique()} unique households")
print(f"  Training: {training['hhid_final'].nunique()} unique households")


print("\n" + "="*80)
print("STANDARDIZING GEOGRAPHIC IDENTIFIERS")
print("="*80)


for col in ['region_name', 'district_name', 'cluster_name']:
    if col not in hh.columns:
        # Try variations
        variations = [c for c in hh.columns if col.replace('_name', '') in c]
        if variations:
            hh[col] = hh[variations[0]]
            log_challenge(
                f"Column '{col}' not found in household list",
                f"Using '{variations[0]}' instead"
            )
        else:
            hh[col] = 'Unknown'
            log_challenge(
                f"Column '{col}' not found in household list",
                f"Setting all values to 'Unknown' - requires data correction"
            )


geo_lookup = hh[['hhid_final', 'region_name', 'district_name', 'cluster_name']].drop_duplicates(subset=['hhid_final'])

print(f"\n✓ Standardized geographic columns")
print(f"  Unique regions: {hh['region_name'].nunique()}")
print(f"  Unique districts: {hh['district_name'].nunique()}")
print(f"  Unique clusters: {hh['cluster_name'].nunique()}")


print("\n" + "="*80)
print("COMBINING COACHING VISIT DATASETS")
print("="*80)

def prepare_coaching(df, source_label):
    df = df.copy()
    
    if 'starttime' in df.columns:
        df['visit_date'] = df['starttime'].apply(excel_serial_to_datetime)
        valid_dates = df['visit_date'].notna().sum()
        print(f"  {source_label}: Parsed {valid_dates}/{len(df)} visit dates from starttime")
    else:
        df['visit_date'] = pd.NaT
        log_challenge(
            f"In {source_label}: 'starttime' column not found",
            "Using file order for visit sequencing"
        )
    df['source_file'] = source_label
    return df

c1p = prepare_coaching(c1, 'coaching1')
c2p = prepare_coaching(c2, 'coaching2')

combined = pd.concat([c1p, c2p], ignore_index=True, sort=False)
print(f"\n✓ Combined coaching files: {len(combined)} total records")
print(f"  From coaching1: {len(c1p)} records")
print(f"  From coaching2: {len(c2p)} records")


if combined['visit_date'].notna().any():
    combined = combined.sort_values(['hhid_final','visit_date']).reset_index(drop=True)
    combined['visit_order'] = combined.groupby('hhid_final').cumcount() + 1
    print(f"  Visit ordering: Based on visit_date")
else:
    if 'visitcount' in combined.columns:
        combined['visitcount'] = pd.to_numeric(combined['visitcount'], errors='coerce')
        combined = combined.sort_values(['hhid_final','visitcount']).reset_index(drop=True)
        combined['visit_order'] = combined.groupby('hhid_final').cumcount() + 1
        print(f"  Visit ordering: Based on visitcount column")
    else:
        combined = combined.reset_index(drop=True)
        combined['visit_order'] = combined.groupby('hhid_final').cumcount() + 1
        print(f"  Visit ordering: Based on file order (fallback)")
        log_challenge(
            "No visit date or visit count columns available",
            "Using file order as fallback - may not reflect actual visit sequence"
        )

combined['visit_count'] = combined.groupby('hhid_final')['visit_order'].transform('max')


combined = combined.merge(
    geo_lookup.rename(columns={
        'region_name': 'region_name_hh',
        'district_name': 'district_name_hh',
        'cluster_name': 'cluster_name_hh'
    }), 
    on='hhid_final', 
    how='left'
)


def coalesce_geo(row, geo_key):
    hh_col = f"{geo_key}_hh"
    coach_col = geo_key if geo_key in row.index else None
    if pd.notna(row.get(hh_col)):
        return row[hh_col]
    elif coach_col and pd.notna(row.get(coach_col)):
        return row[coach_col]
    else:
        return "Unknown"

for g in ['region_name', 'district_name', 'cluster_name']:
    combined[g] = combined.apply(lambda r, g=g: coalesce_geo(r, g), axis=1)


drop_cols = [c for c in ['region_name_hh','district_name_hh','cluster_name_hh'] if c in combined.columns]
combined = combined.drop(columns=drop_cols)


combined.to_csv(OUTPUT_DIR / "Mugume_combined_coaching_visits.csv", index=False)
print(f"\n✓ Saved combined coaching visits to Mugume_combined_coaching_visits.csv")


print("\n" + "="*80)
print("ASSIGNMENT 1: CREATING SUMMARY TABLES")
print("="*80)


total_by_geo = (hh.groupby(['region_name','district_name','cluster_name'], dropna=False)
                  .agg(total_households=('hhid_final','nunique')).reset_index())
total_by_geo.to_csv(OUTPUT_DIR / "Mugume_total_households_by_geo.csv", index=False)
print(f"\n✓ Total households by geography: {len(total_by_geo)} geographic units")


if 'training_type' in training.columns:
    train_counts = (training.groupby(['region_name','district_name','cluster_name','training_type'])
                             .agg(n_households=('hhid_final','nunique')).reset_index())
    train_pivot = (train_counts.pivot_table(index=['region_name','district_name','cluster_name'],
                                           columns='training_type',
                                           values='n_households',
                                           fill_value=0).reset_index())
    print(f"✓ Training participation by type: {training['training_type'].nunique()} training types")
else:
   
    exclude = {'hhid_final','region_name','district_name','cluster_name'}
    indicator_cols = [c for c in training.columns if c not in exclude and training[c].dtype in [np.int64, np.float64]]
    train_pivot = (training.groupby(['region_name','district_name','cluster_name'])[indicator_cols]
                          .agg(lambda s: (s==1).sum()).reset_index())
    print(f"✓ Training participation (indicator columns): {len(indicator_cols)} indicators")
    log_challenge(
        "No 'training_type' column found in training dataset",
        f"Using {len(indicator_cols)} indicator columns as training types"
    )


train_pivot = train_pivot.merge(total_by_geo, on=['region_name','district_name','cluster_name'], how='right')
train_metric_cols = [c for c in train_pivot.columns if c not in ['region_name','district_name','cluster_name','total_households']]
for c in train_metric_cols:
    train_pivot[f"{c}_prop"] = train_pivot[c] / train_pivot['total_households']

train_pivot.to_csv(OUTPUT_DIR / "Mugume_training_counts_and_props_by_geo.csv", index=False)
print(f"✓ Saved training analysis")


visit_counts = combined.groupby('hhid_final', dropna=False).agg(max_visits=('visit_order','max')).reset_index()
visit_counts['visits_1'] = (visit_counts['max_visits'] == 1).astype(int)
visit_counts['visits_2'] = (visit_counts['max_visits'] == 2).astype(int)
visit_counts['visits_3plus'] = (visit_counts['max_visits'] >= 3).astype(int)

visit_geo = visit_counts.merge(geo_lookup, on='hhid_final', how='left')
visits_by_geo = (visit_geo.groupby(['region_name','district_name','cluster_name'])
                 .agg(households_count=('hhid_final','nunique'),
                      visited_once=('visits_1','sum'),
                      visited_twice=('visits_2','sum'),
                      visited_thrice_plus=('visits_3plus','sum'))
                 .reset_index())
visits_by_geo = visits_by_geo.merge(total_by_geo, on=['region_name','district_name','cluster_name'], how='right')
for c in ['visited_once','visited_twice','visited_thrice_plus']:
    visits_by_geo[f"{c}_prop"] = visits_by_geo[c] / visits_by_geo['total_households']

visits_by_geo.to_csv(OUTPUT_DIR / "Mugume_visits_by_geo.csv", index=False)
print(f"✓ Saved visit coverage analysis")


print("\n" + "="*80)
print("ASSIGNMENT 2: CALCULATING ADOPTION SCORES")
print("="*80)


df = combined.copy()
for c in (wash_cols + agri_cols + vsla_cols):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')


def detect_and_rescale(cols, df, component_name):
    if not cols:
        return df
    max_vals = df[cols].max(skipna=True)
    cols_to_rescale = [c for c in cols if pd.notna(max_vals.get(c)) and max_vals[c] > 1.1]
    if cols_to_rescale:
        print(f"  {component_name}: Rescaling {len(cols_to_rescale)} columns from 0-100 to 0-1 scale")
        for c in cols_to_rescale:
            df[c] = df[c] / 100.0
    return df

df = detect_and_rescale(wash_cols, df, "WASH")
df = detect_and_rescale(agri_cols, df, "Agriculture")
df = detect_and_rescale(vsla_cols, df, "VSLA")


df['wash_score'] = safe_mean_row(df, wash_cols) if wash_cols else np.nan
df['agri_score'] = safe_mean_row(df, agri_cols) if agri_cols else np.nan
df['vsla_score'] = safe_mean_row(df, vsla_cols) if vsla_cols else np.nan
df['overall_adoption'] = df[['wash_score','agri_score','vsla_score']].mean(axis=1, skipna=True)

print(f"\n✓ Calculated adoption scores for {len(df)} coaching records")
print(f"  WASH score range: {df['wash_score'].min():.3f} - {df['wash_score'].max():.3f}")
print(f"  Agriculture score range: {df['agri_score'].min():.3f} - {df['agri_score'].max():.3f}")
print(f"  VSLA score range: {df['vsla_score'].min():.3f} - {df['vsla_score'].max():.3f}")
print(f"  Overall adoption range: {df['overall_adoption'].min():.3f} - {df['overall_adoption'].max():.3f}")


df.to_csv(OUTPUT_DIR / "Mugume_coaching_visits_with_scores.csv", index=False)
print(f"✓ Saved scored coaching visits")


hh_scores = (df.groupby('hhid_final')
             .agg(wash_avg=('wash_score','mean'),
                  agri_avg=('agri_score','mean'),
                  vsla_avg=('vsla_score','mean'),
                  overall_avg=('overall_adoption','mean'),
                  n_visits=('visit_order','max'))
             .reset_index())
hh_scores = hh_scores.merge(geo_lookup, on='hhid_final', how='left')
hh_scores.to_csv(OUTPUT_DIR / "Mugume_household_adoption_scores.csv", index=False)
print(f"✓ Saved household-level scores for {len(hh_scores)} households")


first_visit = df[df['visit_order']==1].copy()
first_visit = first_visit.merge(geo_lookup, on='hhid_final', how='left', suffixes=('', '_hh'))


for col in ['region_name', 'district_name', 'cluster_name']:
    if f'{col}_hh' in first_visit.columns:
        first_visit[col] = first_visit[f'{col}_hh'].fillna(first_visit[col])
        first_visit = first_visit.drop(columns=[f'{col}_hh'])

agg_first = (
    first_visit.groupby(['region_name','district_name','cluster_name'])
    .agg(
        wash_mean=('wash_score','mean'),
        agri_mean=('agri_score','mean'),
        vsla_mean=('vsla_score','mean'),
        overall_mean=('overall_adoption','mean'),
        n_households=('hhid_final','nunique')
    )
    .reset_index()
)

agg_first.to_csv(OUTPUT_DIR / "Mugume_first_visit_scores_by_geo.csv", index=False)
print(f"✓ Saved first visit analysis by geography")


print("\n" + "="*80)
print("ANALYZING PROGRAM PERFORMANCE")
print("="*80)


regional_perf = agg_first.groupby('region_name').agg({
    'wash_mean': 'mean',
    'agri_mean': 'mean',
    'vsla_mean': 'mean',
    'overall_mean': 'mean',
    'n_households': 'sum'
}).reset_index()
regional_perf = regional_perf.sort_values('overall_mean', ascending=False)

print("\nRegional Performance (Overall Adoption):")
for idx, row in regional_perf.iterrows():
    print(f"  {row['region_name']}: {row['overall_mean']*100:.1f}% ({row['n_households']} households)")


component_avg = {
    'WASH': agg_first['wash_mean'].mean(),
    'Agriculture': agg_first['agri_mean'].mean(),
    'VSLA': agg_first['vsla_mean'].mean(),
    'Overall': agg_first['overall_mean'].mean()
}
print("\nProgram-wide Component Averages:")
for comp, score in component_avg.items():
    print(f"  {comp}: {score*100:.1f}%")


threshold_low = 0.5
low_performers = agg_first[agg_first['overall_mean'] < threshold_low].sort_values('overall_mean')
print(f"\nClusters Below 50% Threshold: {len(low_performers)}")
if len(low_performers) > 0:
    for idx, row in low_performers.head(10).iterrows():
        print(f"  {row['cluster_name']} ({row['district_name']}): {row['overall_mean']*100:.1f}%")


threshold_high = 0.7
high_performers = agg_first[agg_first['overall_mean'] >= threshold_high].sort_values('overall_mean', ascending=False)
print(f"\nTop Performing Clusters (≥70%): {len(high_performers)}")
if len(high_performers) > 0:
    for idx, row in high_performers.head(10).iterrows():
        print(f"  {row['cluster_name']} ({row['district_name']}): {row['overall_mean']*100:.1f}%")


print("\n" + "="*80)
print("ASSIGNMENT 3: CREATING MONITORING TOOLS")
print("="*80)

def status_from_score(x):
    if pd.isna(x): return "no_data"
    if x >= 0.7: return "green"
    if x >= 0.5: return "amber"
    return "red"

monitor = agg_first.copy()
monitor['status'] = monitor['overall_mean'].apply(status_from_score)
monitor['wash_status'] = monitor['wash_mean'].apply(status_from_score)
monitor['agri_status'] = monitor['agri_mean'].apply(status_from_score)
monitor['vsla_status'] = monitor['vsla_mean'].apply(status_from_score)


monitor['priority'] = monitor['status'].map({
    'red': 'High',
    'amber': 'Medium',
    'green': 'Low',
    'no_data': 'Review'
})

monitor.to_csv(OUTPUT_DIR / "Mugume_activity_monitor.csv", index=False)
print(f"✓ Created activity monitoring tool")
print(f"  Red (High Priority): {(monitor['status']=='red').sum()} clusters")
print(f"  Amber (Medium Priority): {(monitor['status']=='amber').sum()} clusters")
print(f"  Green (On Track): {(monitor['status']=='green').sum()} clusters")


kpi_df = agg_first.copy()
kpi_df['region_cluster'] = kpi_df['region_name'] + " / " + kpi_df['district_name'] + " / " + kpi_df['cluster_name']
kpi_df.to_csv(OUTPUT_DIR / "Mugume_dashboard_kpis.csv", index=False)


print("\n" + "="*80)
print("GENERATING COMPREHENSIVE REPORT")
print("="*80)

doc = Document()


title = doc.add_heading("Program Monitoring Analysis Report", level=0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

subtitle = doc.add_paragraph("Raising The Village - Coaching and Training Program")
subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
subtitle.runs[0].font.size = Pt(14)
subtitle.runs[0].font.color.rgb = RGBColor(128, 128, 128)


doc.add_paragraph()
meta_table = doc.add_table(rows=3, cols=2)
meta_table.style = 'Light Grid Accent 1'
meta_table.cell(0, 0).text = "Prepared by:"
meta_table.cell(0, 1).text = "Mugume Martin"
meta_table.cell(1, 0).text = "Date:"
meta_table.cell(1, 1).text = datetime.now().strftime('%B %d, %Y')
meta_table.cell(2, 0).text = "Analysis Period:"
meta_table.cell(2, 1).text = f"Up to {datetime.now().strftime('%B %Y')}"


doc.add_page_break()
doc.add_heading("Executive Summary", level=1)
doc.add_paragraph(
    f"This report presents a comprehensive analysis of the Raising The Village program, "
    f"covering {hh['hhid_final'].nunique()} households across {hh['region_name'].nunique()} regions, "
    f"{hh['district_name'].nunique()} districts, and {hh['cluster_name'].nunique()} clusters."
)

summary_para = doc.add_paragraph()
summary_para.add_run("Key Findings:\n").bold = True
doc.add_paragraph(
    f"• Overall program adoption rate: {component_avg['Overall']*100:.1f}%",
    style='List Bullet'
)
doc.add_paragraph(
    f"• WASH component average: {component_avg['WASH']*100:.1f}%",
    style='List Bullet'
)
doc.add_paragraph(
    f"• Agriculture component average: {component_avg['Agriculture']*100:.1f}%",
    style='List Bullet'
)
doc.add_paragraph(
    f"• VSLA participation average: {component_avg['VSLA']*100:.1f}%",
    style='List Bullet'
)
doc.add_paragraph(
    f"• {len(low_performers)} clusters require immediate attention (below 50% threshold)",
    style='List Bullet'
)



doc.add_page_break()
doc.add_heading("1. Data Processing & Challenges", level=1)


if CHALLENGES_LOG:
    doc.add_heading("Challenges Encountered & Solutions", level=2)
    challenge_table = doc.add_table(rows=1 + len(CHALLENGES_LOG), cols=2)
    challenge_table.style = 'Table Grid'
    hdr_cells = challenge_table.rows[0].cells
    hdr_cells[0].text = 'Challenge'
    hdr_cells[1].text = 'Solution'
    for i, log in enumerate(CHALLENGES_LOG, 1):
        row = challenge_table.rows[i].cells
        row[0].text = log['challenge']
        row[1].text = log['solution']
else:
    doc.add_paragraph("No significant data quality issues encountered.")

doc.add_page_break()
doc.add_heading("2. Assignment 1: Summary Tables", level=1)


doc.add_heading("Total Households by Geography", level=2)
total_table = doc.add_table(rows=1 + len(total_by_geo), cols=4)
total_table.style = 'Light List Accent 1'
hdr = total_table.rows[0].cells
hdr[0].text = 'Region'; hdr[1].text = 'District'; hdr[2].text = 'Cluster'; hdr[3].text = 'Total Households'
for i, row in enumerate(total_by_geo.itertuples(), 1):
    cells = total_table.rows[i].cells
    cells[0].text = str(row.region_name)
    cells[1].text = str(row.district_name)
    cells[2].text = str(row.cluster_name)
    cells[3].text = str(row.total_households)


doc.add_page_break()
doc.add_heading("Training Attendance", level=2)


numeric_cols = []
for c in train_pivot.columns:
    if c not in ['region_name', 'district_name', 'cluster_name']:
        # Check if column is numeric
        if pd.api.types.is_numeric_dtype(train_pivot[c]) or train_pivot[c].dtype in ['float64', 'int64']:
            numeric_cols.append(c)

train_sample = train_pivot.head(10).copy()


train_table = doc.add_table(rows=1 + len(train_sample), cols=len(train_sample.columns))
train_table.style = 'Light List Accent 2'


for j, col_name in enumerate(train_sample.columns):
    clean_name = col_name.replace('_prop', ' (Prop)')
    train_table.rows[0].cells[j].text = clean_name


for i, (_, row) in enumerate(train_sample.iterrows(), 1):  # Use iterrows() → safe indexing
    for j, col_name in enumerate(train_sample.columns):
        val = row.iloc[j]  # Safe: use .iloc[j] instead of getattr
        cell = train_table.rows[i].cells[j]

        if col_name in numeric_cols:
            if pd.isna(val):
                cell.text = "N/A"
            elif '_prop' in col_name:
                cell.text = f"{val:.1%}"
            else:
                cell.text = f"{val:,.0f}"
        else:
            cell.text = str(val) if not pd.isna(val) else "Unknown"


doc.add_page_break()
doc.add_heading("Coaching Visit Coverage", level=2)
visit_sample = visits_by_geo[['region_name', 'district_name', 'cluster_name',
                              'visited_once_prop', 'visited_twice_prop', 'visited_thrice_plus_prop']].head(10)
visit_table = doc.add_table(rows=1 + len(visit_sample), cols=6)
visit_table.style = 'Light List Accent 3'
hdr = visit_table.rows[0].cells
hdr[0].text = 'Region'; hdr[1].text = 'District'; hdr[2].text = 'Cluster'
hdr[3].text = '1 Visit (%)'; hdr[4].text = '2 Visits (%)'; hdr[5].text = '3+ Visits (%)'
for i, row in enumerate(visit_sample.itertuples(), 1):
    cells = visit_table.rows[i].cells
    cells[0].text = str(row.region_name)
    cells[1].text = str(row.district_name)
    cells[2].text = str(row.cluster_name)
    cells[3].text = f"{row.visited_once_prop:.1%}"
    cells[4].text = f"{row.visited_twice_prop:.1%}"
    cells[5].text = f"{row.visited_thrice_plus_prop:.1%}"


doc.add_page_break()
doc.add_heading("3. Assignment 2: Adoption Scores & Performance", level=1)

doc.add_heading("First Visit Adoption Scores by Cluster (Sample)", level=2)
score_sample = agg_first[['region_name', 'district_name', 'cluster_name',
                          'wash_mean', 'agri_mean', 'vsla_mean', 'overall_mean']].head(15)
score_table = doc.add_table(rows=1 + len(score_sample), cols=7)
score_table.style = 'Light List Accent 4'
hdr = score_table.rows[0].cells
hdr[0].text = 'Region'; hdr[1].text = 'District'; hdr[2].text = 'Cluster'
hdr[3].text = 'WASH'; hdr[4].text = 'Agriculture'; hdr[5].text = 'VSLA'; hdr[6].text = 'Overall'
for i, row in enumerate(score_sample.itertuples(), 1):
    cells = score_table.rows[i].cells
    cells[0].text = str(row.region_name)
    cells[1].text = str(row.district_name)
    cells[2].text = str(row.cluster_name)
    cells[3].text = f"{row.wash_mean:.1%}"
    cells[4].text = f"{row.agri_mean:.1%}"
    cells[5].text = f"{row.vsla_mean:.1%}"
    cells[6].text = f"{row.overall_mean:.1%}"

doc.add_paragraph()
doc.add_heading("Performance Commentary", level=2)
perf_para = doc.add_paragraph()
perf_para.add_run("Key Observations:\n").bold = True
doc.add_paragraph(
    f"• Program-wide first-visit adoption stands at {component_avg['Overall']*100:.1f}%, "
    f"with WASH ({component_avg['WASH']*100:.1f}%) and Agriculture ({component_avg['Agriculture']*100:.1f}%) "
    f"leading, while VSLA participation lags at {component_avg['VSLA']*100:.1f}%.",
    style='List Bullet'
)
doc.add_paragraph(
    f"• {len(low_performers)} clusters fall below 50% adoption, primarily in WASH and VSLA components.",
    style='List Bullet'
)
doc.add_paragraph(
    f"• Top performers exceed 80% adoption, showing strong integration of all components.",
    style='List Bullet'
)


doc.add_page_break()
doc.add_heading("4. Strategic Recommendations", level=1)
rec_para = doc.add_paragraph()
rec_para.add_run("Actionable Recommendations:\n").bold = True
doc.add_paragraph(
    "1. Target low-performing clusters with intensive WASH and VSLA follow-up coaching in the next quarter.",
    style='List Bullet'
)
doc.add_paragraph(
    "2. Replicate success factors from top clusters (e.g., integrated training sessions) across underperforming areas.",
    style='List Bullet'
)
doc.add_paragraph(
    "3. Standardize data collection: enforce 'hhid', 'starttime', and 0–1 scoring to reduce cleaning overhead.",
    style='List Bullet'
)


doc.add_page_break()
doc.add_heading("5. Assignment 3: Activity Monitoring Tool", level=1)
doc.add_paragraph(
    "An interactive monitoring dashboard has been developed to enable real-time program oversight. "
    "The tool includes:"
)
doc.add_paragraph("• Traffic-light status for each cluster (Red/Amber/Green)", style='List Bullet')
doc.add_paragraph("• Filterable views by region and district", style='List Bullet')
doc.add_paragraph("• KPI cards and trend indicators", style='List Bullet')
doc.add_paragraph("• Exportable data tables", style='List Bullet')

doc.add_paragraph()
doc.add_paragraph("Run the dashboard using:")
code_para = doc.add_paragraph()
code_para.add_run("python ").italic = True
code_para.add_run(f"\"{DASH_APP_PATH.name}\"").font.highlight_color = 3  # Yellow


doc.save(REPORT_PATH)
print(f"Comprehensive Word report saved to: {REPORT_PATH.name}")


print("\nGenerating interactive dashboard app...")

dash_code = f'''# {DASH_APP_PATH.name} - Raising The Village Program Monitoring Dashboard
import dash
from dash import dcc, html, dash_table, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Load data - NOTE: Double curly braces {{}} to escape f-string
df = pd.read_csv(r"{(OUTPUT_DIR / 'Mugume_first_visit_scores_by_geo.csv').as_posix()}")
training = pd.read_csv(r"{(OUTPUT_DIR / 'Mugume_training_counts_and_props_by_geo.csv').as_posix()}")
visits = pd.read_csv(r"{(OUTPUT_DIR / 'Mugume_visits_by_geo.csv').as_posix()}")

# Convert to percentage
for col in ['wash_mean', 'agri_mean', 'vsla_mean', 'overall_mean']:
    if col in df.columns and df[col].max() <= 1.1:
        df[col] = df[col] * 100

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Raising The Village - Program Monitoring"

# Layout
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Program Monitoring Dashboard", className="text-center text-primary mb-4"))),
    
    dbc.Row([
        dbc.Col(dcc.Dropdown(
            id='region-filter',
            options=[{{'label': r, 'value': r}} for r in sorted(df['region_name'].dropna().unique())],
            placeholder="All Regions", className="mb-3"
        ), width=4),
        dbc.Col(dcc.Dropdown(id='district-filter', placeholder="All Districts"), width=4),
    ]),
    
    dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody([
            html.H4(id='kpi-overall'), html.P("Overall Adoption")
        ]), color="success", inverse=True), width=3),
        dbc.Col(dbc.Card(dbc.CardBody([
            html.H4(id='kpi-wash'), html.P("WASH Score")
        ]), color="info", inverse=True), width=3),
        dbc.Col(dbc.Card(dbc.CardBody([
            html.H4(id='kpi-agri'), html.P("Agriculture Score")
        ]), color="warning", inverse=True), width=3),
        dbc.Col(dbc.Card(dbc.CardBody([
            html.H4(id='kpi-vsla'), html.P("VSLA Participation")
        ]), color="danger", inverse=True), width=3),
    ], className="mb-4"),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='main-chart'), width=8),
        dbc.Col(dcc.Graph(id='radar-chart'), width=4),
    ]),
    
    dbc.Row([
        dbc.Col(dcc.Graph(id='visits-chart'), width=6),
        dbc.Col(dcc.Graph(id='training-chart'), width=6),
    ]),
    
    dbc.Row(dbc.Col(dash_table.DataTable(
        id='data-table', page_size=10, style_table={{'overflowX': 'auto'}}
    ))),
    
    html.Footer("© 2025 Raising The Village | Prepared by: Mugume Martin", className="text-center text-muted mt-4")
], fluid=True)

# Callbacks
@app.callback(
    Output('district-filter', 'options'),
    Input('region-filter', 'value')
)
def update_districts(region):
    if region:
        districts = sorted(df[df['region_name'] == region]['district_name'].unique())
    else:
        districts = sorted(df['district_name'].unique())
    return [{{'label': d, 'value': d}} for d in districts]

@app.callback(
    [Output('kpi-overall', 'children'), Output('kpi-wash', 'children'),
     Output('kpi-agri', 'children'), Output('kpi-vsla', 'children'),
     Output('main-chart', 'figure'), Output('radar-chart', 'figure'),
     Output('visits-chart', 'figure'), Output('training-chart', 'figure'),
     Output('data-table', 'data'), Output('data-table', 'columns')],
    [Input('region-filter', 'value'), Input('district-filter', 'value')]
)
def update_dashboard(region, district):
    filtered = df.copy()
    if region: filtered = filtered[filtered['region_name'] == region]
    if district: filtered = filtered[filtered['district_name'] == district]
    
    kpi_o = f"{{filtered['overall_mean'].mean():.1f}}%" if not filtered.empty else "N/A"
    kpi_w = f"{{filtered['wash_mean'].mean():.1f}}%" if not filtered.empty else "N/A"
    kpi_a = f"{{filtered['agri_mean'].mean():.1f}}%" if not filtered.empty else "N/A"
    kpi_v = f"{{filtered['vsla_mean'].mean():.1f}}%" if not filtered.empty else "N/A"
    
    bar_fig = px.bar(filtered.sort_values('overall_mean'), 
                     x='cluster_name', y='overall_mean', color='overall_mean',
                     color_continuous_scale=['red', 'orange', 'green'], range_color=[0, 100],
                     labels={{'overall_mean': 'Overall Adoption (%)'}}, height=500)
    bar_fig.update_layout(xaxis_title="Cluster", yaxis_title="Adoption (%)")
    
    radar_fig = go.Figure()
    means = [filtered[c].mean() for c in ['wash_mean','agri_mean','vsla_mean','overall_mean']]
    radar_fig.add_trace(go.Scatterpolar(r=means, theta=['WASH','Agriculture','VSLA','Overall'], 
                                       fill='toself', name='Selected'))
    radar_fig.update_layout(polar=dict(radialaxis=dict(range=[0,100])), height=400)
    
    visits_fig = px.bar(visits[visits['region_name'].isin(filtered['region_name'].unique())] if region else visits,
                        x='cluster_name', y=['visited_once_prop','visited_twice_prop','visited_thrice_plus_prop'],
                        barmode='stack', labels={{'value': 'Proportion', 'variable': 'Visit Count'}})
    visits_fig.update_yaxes(tickformat='.0%')
    
    train_cols = [c for c in training.columns if '_prop' in c]
    train_fig = px.bar(training[training['region_name'].isin(filtered['region_name'].unique())] if region else training,
                       x='cluster_name', y=train_cols, barmode='group')
    train_fig.update_yaxes(tickformat='.0%')
    
    table_data = filtered[['region_name','district_name','cluster_name','overall_mean']].round(1).to_dict('records')
    table_cols = [
        {{"name": "Region", "id": "region_name"}},
        {{"name": "District", "id": "district_name"}},
        {{"name": "Cluster", "id": "cluster_name"}},
        {{"name": "Overall (%)", "id": "overall_mean"}}
    ]
    
    return kpi_o, kpi_w, kpi_a, kpi_v, bar_fig, radar_fig, visits_fig, train_fig, table_data, table_cols

if __name__ == '__main__':
    print("Starting dashboard...")
    print("Open http://127.0.0.1:8050 in your browser")
    app.run(debug=True)
'''


with open(DASH_APP_PATH, 'w', encoding='utf-8') as f:
    f.write(dash_code)

print(f"Interactive dashboard saved: {DASH_APP_PATH.name}")
print(f"   Run: python \"{DASH_APP_PATH.name}\"")


print("\n" + "="*80)
print("ANALYSIS COMPLETE - ALL ASSIGNMENTS FINISHED")
print("="*80)
print(f"Outputs saved in: {OUTPUT_DIR}")
print("\nGenerated Files:")
for f in sorted(OUTPUT_DIR.iterdir()):
    if f.is_file():
        print(f"  • {f.name}")
print(f"\nWord Report: {REPORT_PATH.name}")
print(f"Dashboard: {DASH_APP_PATH.name}")


PROGRAM MONITORING DATA ANALYSIS - RAISING THE VILLAGE

Loading files from: E:\MUGUME\Raising village
  ✓ households: household_list.xlsx
  ✓ coaching1: households_coaching_visits.xlsx
  ✓ coaching2: households_coaching_visits data 2.xlsx
  ✓ training: households_training_attendance..xlsx

✓ Files loaded successfully.

IDENTIFYING COACHING COMPONENT COLUMNS (H:AB for WASH, AC:AP for AGRI)

Checking column positions in 'households_coaching_visits.xlsx':
  Column H (index 7): latrine_present
  Column AB (index 27): dishrack_double_pres
  Column AC (index 28): compost_present
  Column AP (index 41): bbw_management

✓ Identified 21 WASH columns (H:AB)
✓ Identified 14 Agriculture columns (AC:AP)
✓ Identified 1 VSLA column(s)

✓ Saved column mapping to Mugume_column_mapping.csv

STANDARDIZING HOUSEHOLD IDENTIFIERS

✓ Created hhid_final in all datasets
  Household list: 2556 unique households
  Coaching 1: 1885 unique households
  Coaching 2: 643 unique households
  Training: 1684 unique hous

In [32]:
ls

 Volume in drive E has no label.
 Volume Serial Number is 0DD0-07C4

 Directory of E:\MUGUME\Raising village

11/14/2025  12:38 PM    <DIR>          .
11/14/2025  09:07 AM    <DIR>          ..
11/11/2025  01:23 AM            96,888 household_list.xlsx
11/11/2025  01:18 AM           378,489 households_coaching_visits data 2.xlsx
11/11/2025  01:12 AM           900,937 households_coaching_visits.xlsx
11/11/2025  01:19 AM           643,823 households_training_attendance..xlsx
11/14/2025  11:03 AM               467 Mugume_first_visit_scores_by_geo_v3.csv
11/14/2025  12:39 PM    <DIR>          output_mugume
11/14/2025  01:13 PM    <DIR>          outputs
11/14/2025  09:22 AM         2,524,967 Program Monitoring - Data Analyst Interview Questions Nov 2025.docx
11/14/2025  09:10 AM    <DIR>          scheduledwrittenassessmentdataanalystprogrammonitori
11/14/2025  09:07 AM         1,659,039 scheduledwrittenassessmentdataanalystprogrammonitori.zip
               7 File(s)      6,204,610 bytes
   